In [1]:
# DIRECTORY SET
import os
import sys
import traceback
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent.parent
os.chdir(base_dir)
print(os.getcwd())

# ENVIRONMENT VARIABLES
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

# Enable autoreload
%load_ext autoreload
%autoreload 2

# Import custom modules
from destinyapp.models import StreamRecapData

from core import services
from core import utils
from core import controller

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
video_id="OqVH_MTBQ6k"

In [6]:
stream_recap_data=await utils.get_recap_data(video_id)

transcript, linked_transcript=await services.process_raw_transcript(stream_recap_data.raw_transcript_data, video_id)
stream_recap_data.transcript=transcript
stream_recap_data.linked_transcript=linked_transcript

await sync_to_async(stream_recap_data.save)()

{'text': 'looks', 'start': 186935, 'end': 187351, 'confidence': 0.96389, 'speaker': 'A', 'channel': None}
48
4668
Destiny starts
Finished diarization cutoff


In [7]:
stream_recap_data=await utils.get_recap_data(video_id)

annotated_results, major_topics, minor_topics, cost = await services.stream_plot.generate_data(stream_recap_data)

Cost:  0.007353
Cost:  0.0023335
Cost:  0.0024312500000000003
Cost:  0.00247275
Cost:  0.00172625
Cost:  0.00271075
Cost:  0.0027155
Cost:  0.00268825
Cost:  0.0028465
Cost:  0.0026525
Cost:  0.00290175
Cost:  0.0028795
Cost:  0.00257375
Cost:  0.002435
Cost:  0.00278025
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  13


In [8]:
plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data, annotated_results, major_topics, minor_topics, video_id)

In [30]:
plot_object, cost=await services.stream_plot.annotate_extra(video_id, stream_recap_data, plot_object)      

Counters:  [105, 68]


In [31]:
from dataclasses import dataclass, asdict, is_dataclass
stream_recap_data.plot_object=asdict(plot_object)
await sync_to_async(stream_recap_data.save)()

In [23]:
context_offset=500

tasks=[]
for i in range(len(plot_object.segments)):
    task_text=None
    if plot_object.segments[i].texts!=[]:
        start_index=stream_recap_data.transcript.find(plot_object.segments[i].texts[0])
        contextualized_start_index=start_index-context_offset
        if (contextualized_start_index)<0:
            contextualized_start_index=0
        task_text=stream_recap_data.transcript[contextualized_start_index:start_index+len(plot_object.segments[i].texts[0])]
    tasks.append(services.stream_plot.extra_annotations.annotate_start(stream_recap_data, plot_object.segments[i].category, contextualized_start_index, task_text))
# start_counts_and_costs=await asyncio.gather(*tasks)


In [24]:
test_index=69
real_start_character_index, cost, response=await tasks[test_index]

In [22]:
real_start_character_index

-1

In [ ]:
stream_recap_data.transcript[160971:]

In [25]:
response

'"he said he took a bullet for."'

In [27]:
plot_object.segments[test_index].category

"Trump's Actions and Republican Party"

In [26]:
plot_object.segments[test_index].texts[0]

"dude behind him that got fucking killed. It's unhinged. \n\nC: But does it matter to you? \n\nDestiny: Does this? It does matter to me, yeah. Cause it's weird when people say God protected Donald Trump that day and it's like, bro, what about the dude behind him? \n\nB: The GOTy killed were leaving out a critical part of the claim that Donald Trump took a bullet because he actually said this at a rally. I think yesterday I saw a brief clip. He said he took a bullet for. \n\nDestiny: Democracy'S a direct quote. Apparently God can only shield one person. Say wor at a time. \n\nC: You're in enough trouble. \n\nDestiny: I will take a trouble. I don't give a fuck. Okay? I will do care. I'll take it all to him. Bolden the liberals around me to be brave. Okay, you see these Laura loomer tweets? You see these unhinged fucking statementerv are making? Yeah. Tell me, where is all the, where's all the conviction for the people calling those people? Fuck that bullshit. I have. \n\nC: When they go 

In [ ]:
task_text=None
if plot_object.segments[test_index].texts!=[]:
    start_index=stream_recap_data.transcript.find(plot_object.segments[test_index].texts[0])
    contextualized_start_index=start_index-context_offset
    if (contextualized_start_index)<0:
        contextualized_start_index=0
    task_text=stream_recap_data.transcript[contextualized_start_index:start_index+len(plot_object.segments[test_index].texts[0])]

In [ ]:
print(task_text)

In [ ]:
# conver the character indexes to start times
from bs4 import BeautifulSoup
soup = BeautifulSoup(stream_recap_data.linked_transcript, 'html.parser')
soup_list=soup.find_all('a')

# Setup list for getting the time at each character count
transcript_soup_character_counter=[]
character_count_total=0
temp_time=0
for link in soup_list:
    link_text = link.get_text()
    character_count_total+=len(link_text)

    if link.get('href') is not None:
        temp_time=int(link['href'].split("t=")[-1].split("s")[0])

    transcript_soup_character_counter.append([character_count_total, temp_time])

In [ ]:
def find_nearest_time_at_character_count(transcript_soup_character_counter, character_count):
    # middle sort the list to get to the closest character count quickly
    l=0
    r=len(transcript_soup_character_counter)-1

    while l<r:
        m=(l+r)//2
        if transcript_soup_character_counter[m][0]<character_count:
            l=m+1
        else:
            r=m

    if l==0:
        return transcript_soup_character_counter[l][1]
    else:
        if abs(transcript_soup_character_counter[l][0]-character_count)<abs(transcript_soup_character_counter[l-1][0]-character_count):
            return transcript_soup_character_counter[l][1]
        else:
            return transcript_soup_character_counter[l-1][1]

In [ ]:
time=find_nearest_time_at_character_count(transcript_soup_character_counter, real_start_character_index)

In [ ]:
time

In [ ]:
print(stream_recap_data.transcript)#[160971:])